### 作業目的: 熟練以Torchtext進行文本資料讀取

本次作業主要會使用[polarity](http://www.cs.cornell.edu/people/pabo/movie-review-data/)的電影評論來進行使用torchtext資料讀取，學員可以在附件的polarity.tsv看到所使用的資料。

Hint: 這次作業同學可以嘗試使用[torchtext.data.TabularDataset](https://torchtext.readthedocs.io/en/latest/data.html#tabulardataset)，可以更簡易讀取資料

### 載入套件

In [1]:
import torch
import pandas as pd
import numpy as np
from torchtext import data, datasets

In [2]:
# 探索資料
# 可以發現資料為文本與類別，而類別即為正評與負評
input_data = pd.read_csv('./polarity.tsv', delimiter='\t', header=None, names=['text', 'label'])
input_data

,text,label
0,films adapted from comic books have had plenty...,1
1,every now and then a movie comes along from a ...,1
2,you've got mail works alot better than it dese...,1
3,jaws is a rare film that grabs your attentio...,1
4,moviemaking is a lot like being the general ma...,1
...,...,...
1995,"if anything , "" stigmata "" should be taken as ...",0
1996,"john boorman's "" zardoz "" is a goofy cinematic...",0
1997,the kids in the hall are an acquired taste .it...,0
1998,there was a time when john carpenter was a gre...,0


### 建立Pipeline生成資料

In [5]:
# 建立Field與Dataset

text_field = data.Field(lower=True, tokenize='spacy', dtype=torch.float64)
label_field = data.Field(sequential=False)
input_data = data.TabularDataset(path='./polarity.tsv', format='tsv', fields=[('text', text_field), ('label', label_field)])

In [6]:
# 取的examples並打亂順序
examples = input_data.examples
np.random.shuffle(examples)

# 以8:2的比例切分examples
train_ex = examples[:int(len(examples)*0.8)]
test_ex = examples[int(len(examples)*0.8):]

# 建立training與testing dataset
train_data = data.Dataset(examples=train_ex, fields={'text':text_field, 'label':label_field})
test_data = data.Dataset(examples=test_ex, fields={'text':text_field, 'label':label_field})

train_data[0].label, train_data[0].text

('0',
 [' ',
  'tina',
  '!',
  '!',
  '!',
  'fetch',
  'me',
  'the',
  'axe',
  '!',
  '!',
  '!',
  'a',
  'favourite',
  'book',
  'of',
  'mine',
  'called',
  'the',
  'golden',
  'turkey',
  'awards',
  'relates',
  'the',
  'story',
  'that',
  'when',
  'mommie',
  'dearest',
  'was',
  'unleashed',
  'upon',
  'unsuspecting',
  'audiences',
  'back',
  'in',
  '1981',
  ',',
  'paramount',
  'soon',
  'realised',
  'they',
  'had',
  'a',
  'problem',
  'on',
  'their',
  'hands',
  '.it',
  'was',
  "n't",
  'just',
  'the',
  'film',
  "'s",
  'disappointing',
  'box',
  'office',
  'performance',
  '.indeed',
  ',',
  'in',
  'the',
  'coming',
  'years',
  'some',
  'people',
  'would',
  'be',
  'going',
  'back',
  'to',
  'see',
  'it',
  'two',
  ',',
  'three',
  ',',
  'even',
  'six',
  'times',
  '.no',
  ',',
  'the',
  'main',
  'problem',
  'was',
  'that',
  'what',
  'was',
  'intended',
  'as',
  'a',
  'serious',
  'biopic',
  'of',
  'screen',
  'queen',


In [8]:
# 建立字典
text_field.build_vocab(train_data)
label_field.build_vocab(train_data)
print(f"Vocabularies of index 0-10: {text_field.vocab.itos[:10]} \n")
print(f"words to index {text_field.vocab.stoi}")

Vocabularies of index 0-10: ['<unk>', '<pad>', ',', 'the', 'a', 'and', 'of', 'to', 'is', 'in'] 

words to index defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x00000221E7F93248>>, {'<unk>': 0, '<pad>': 1, ',': 2, 'the': 3, 'a': 4, 'and': 5, 'of': 6, 'to': 7, 'is': 8, 'in': 9, "'s": 10, '"': 11, 'that': 12, '-': 13, 'it': 14, ')': 15, '(': 16, '.': 17, 'as': 18, 'with': 19, 'for': 20, 'film': 21, 'his': 22, 'this': 23, 'he': 24, '.the': 25, 'are': 26, 'on': 27, 'but': 28, 'i': 29, 'be': 30, "n't": 31, 'by': 32, 'movie': 33, 'who': 34, 'an': 35, 'not': 36, 'one': 37, 'was': 38, 'have': 39, 'you': 40, 'from': 41, 'has': 42, 'at': 43, 'her': 44, 'all': 45, 'they': 46, '?': 47, 'out': 48, 'like': 49, 'about': 50, 'up': 51, 'more': 52, 'so': 53, 'which': 54, ':': 55, 'their': 56, 'or': 57, 'do': 58, 'what': 59, '.it': 60, 'does': 61, 'just': 62, 'some': 63, 'when': 64, 'him': 65, 'into': 66, 'there': 67, 'she': 68, "'": 69, 'than': 70, 'only': 71, 'ti

In [13]:
# create iterator for training and testing data
train_iter, test_iter = data.Iterator.splits(datasets=(train_data, test_data), batch_size=3, repeat=False, sort_key= lambda ex:len(ex.text))

In [14]:
for train_batch in train_iter:
    print(train_batch.text, train_batch.text.shape)
    print(train_batch.label, train_batch.label.shape)
    break

tensor([[3.8450e+03, 1.6000e+01, 2.3350e+03],
        [5.5000e+01, 3.5780e+03, 5.5000e+01],
        [2.2897e+04, 1.1400e+02, 3.3830e+03],
        ...,
        [1.0000e+00, 5.6000e+01, 1.0000e+00],
        [1.0000e+00, 4.6760e+03, 1.0000e+00],
        [1.0000e+00, 1.7000e+01, 1.0000e+00]], dtype=torch.float64) torch.Size([1124, 3])
tensor([1, 1, 1]) torch.Size([3])
